 ## Gesture volume controller  (AI)

In [1]:
 #pip install pycaw
 #pip install media pipe 
 # pip install math

In [2]:
import cv2
import mediapipe as mp
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np
 

In [3]:
 
cap = cv2.VideoCapture(0) #Checks for camera
 
mpHands = mp.solutions.hands #detects hand/finger
hands = mpHands.Hands()   #complete the initialization configuration of hands
mpDraw = mp.solutions.drawing_utils
 
#To access speaker through the library pycaw 
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volbar=400
volper=0
 
volMin,volMax = volume.GetVolumeRange()[:2]
 
while True:
    success,img = cap.read() #If camera works capture an image
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Convert to rgb
    
    #Collection of gesture information
    results = hands.process(imgRGB) #completes the image processing.
 
    lmList = [] #empty list
    if results.multi_hand_landmarks: #list of all hands detected.
        #By accessing the list, we can get the information of each hand's corresponding flag bit
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark): #adding counter and returning it
                # Get finger joint points
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy]) #adding to the empty list 'lmList'
            mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)
    
    if lmList != []:
        #getting the value at a point
                        #x      #y
        x1,y1 = lmList[4][1],lmList[4][2]  #thumb
        x2,y2 = lmList[8][1],lmList[8][2]  #index finger
        #creating circle at the tips of thumb and index fingerp
        cv2.circle(img,(x1,y1),13,(0,255,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.circle(img,(x2,y2),13,(0,255,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.line(img,(x1,y1),(x2,y2),(255,0,255),3)  #create a line b/w tips of index finger and thumb
 
        length = hypot(x2-x1,y2-y1) #distance b/w tips using hypotenuse
 # from numpy we find our length,by converting hand range in terms of volume range ie b/w -63.5 to 0
        vol = np.interp(length,[30,350],[volMin,volMax]) 
        volbar=np.interp(length,[30,350],[400,150])
        volper=np.interp(length,[30,350],[0,100])
        
        
        print(vol,int(length))
        volume.SetMasterVolumeLevel(vol, None)
        
        # Hand range 30 - 350
        # Volume range -63.5 - 0.0
        #creating volume bar for volume level 
        cv2.rectangle(img,(50,150),(85,400),(255,0,0),4) # vid ,initial position ,ending position ,rgb ,thickness
        cv2.rectangle(img,(50,int(volbar)),(85,400),(255,0,0),cv2.FILLED)
        cv2.putText(img,f"{int(volper)}%",(10,40),cv2.FONT_ITALIC,1,(255,0,0),3)
        #tell the volume percentage ,location,font of text,length,rgb color,thickness
    cv2.imshow('Image',img) #Show the video 
    if cv2.waitKey(1) & 0xff==ord('p'): #By using spacebar delay will stop
        break
        
cap.release(p)     #stop cam       
cv2.destroyAllWindows() #close window

-65.25 18
-65.25 26
-65.1656210140319 30
-65.25 13
-65.25 14
-65.25 19
-65.25 15
-65.25 20
-65.25 18
-65.25 18
-65.25 16
-65.25 20
-65.25 15
-65.25 18
-65.25 20
-65.25 25
-64.32429986527576 34
-63.672596639166336 37
-65.25 19
-65.25 19
-65.25 19
-64.9936900035767 31
-65.25 29
-57.69178041160943 67
-56.673198885889754 72
-52.40390625 93
-52.198915422972675 94
-47.71318361007865 116
-48.12098069154531 114
-43.635187854801515 136
-44.04298916637646 134
-39.9649629736994 154
-39.95966726565949 154
-37.9265625 164
-37.31423325771797 167
-37.309349713416054 167
-37.110330641536045 168
-35.255050974740996 177
-35.05126112066799 178
-31.96049120339712 193
-31.73377070241893 194
-30.268063074235734 201
-30.689551549415064 199
-29.57088248052191 204
-28.94194527348447 208
-28.371969934315935 210
-28.203552674630032 211
-27.724580679062356 214
-27.79704507664986 213
-28.641164078757313 209
-28.610035420434663 209
-30.553325539570423 200
-30.73864332617351 199
-34.00724421111333 183
-35.0575585643

NameError: name 'p' is not defined